In [1]:
import numpy as np
import tensorflow as tf
from Disturbance import cal_disturbance
from Dynamics import impedance_matrix
import pandas as pd

/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd], sort=False)
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/converter/pypower/from_ppc.py:277: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup["element"].loc[~is_line] = idx_trafo
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation wit

In [2]:
KK = np.load('Controller_K.npy')
V0 = np.load('Voltage0.npy') - 11

filename = 'case34sa_mod.mat'

matrix_R, matrix_X = impedance_matrix(filename)

B = np.hstack([matrix_R, matrix_X])
A = np.eye(B.shape[0])


/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd], sort=False)
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/converter/pypower/from_ppc.py:277: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup["element"].loc[~is_line] = idx_trafo


### Find proper number of past steps $H = 16$ :  $(A + B \mathbb{K})^{H+1}$ is so small that the state at $t-H$ can be regarded

In [3]:
close = A + B @ KK

print(close)

print("\n")
for k in range(15):
    close = close @ close

# Compute the eigenvalues
eigenvalues = np.linalg.eigvals(close)

# Compute the spectral radius
spectral_radius = max(eigenvalues)

print(np.max(close))

[[ 9.84812523e-01 -2.39200641e-02 -2.35914904e-02 ...  7.62467046e-04
   1.01210280e-03  1.09189829e-03]
 [-2.39143798e-02  9.57482573e-01 -4.30983119e-02 ...  1.35045646e-03
   1.84607945e-03  2.00507663e-03]
 [-2.35917351e-02 -4.30988625e-02  9.42620025e-01 ...  9.78957926e-04
   2.16015864e-03  2.54919247e-03]
 ...
 [ 7.63474283e-04  1.35840325e-03  9.81758405e-04 ...  8.40078692e-01
  -1.73286692e-01 -1.77271000e-01]
 [ 1.02482208e-03  1.87653460e-03  2.16724195e-03 ... -1.73049357e-01
   7.98824771e-01 -2.09985312e-01]
 [ 1.10888906e-03  2.04364285e-03  2.55537457e-03 ... -1.76922208e-01
  -2.09875903e-01  7.76920342e-01]]


0.00017142378182987612


### check the disturbances

In [4]:
dis_list = []
for i in range(500):
    dis1 = cal_disturbance(i)
    maxd = np.max(np.abs(dis1))
    dis_list.append(maxd)


print(dis_list.index(max(dis_list)))

# maximum positive dis happen on 157: load decrease dramatically
# maximum negative dis happen on 164: load increase dramatically, voltage drop

164


In [5]:
dis = cal_disturbance(374)

print(dis)

[-0.00493109 -0.00922715 -0.01512806 -0.02019698 -0.02497037 -0.02826562
 -0.03006377 -0.03216835 -0.03317632 -0.03356845 -0.0336846  -0.00952235
 -0.00979168 -0.00986421 -0.00986929 -0.02872675 -0.03184541 -0.0352994
 -0.03807437 -0.04048392 -0.04324666 -0.04551554 -0.04764631 -0.04867215
 -0.04906637 -0.0491842  -0.02856462 -0.02876381 -0.02886362 -0.0334621
 -0.03374806 -0.03389111 -0.03393864]


In [6]:
u1 = KK @ V0

print(u1)

[-0.00405868 -0.00753662 -0.00830913 -0.00882463 -0.00924935 -0.01017957
 -0.0117166  -0.01481702 -0.01722985 -0.01753805 -0.01770522 -0.01139828
 -0.01536268 -0.01668785 -0.01702737 -0.00898565 -0.00852301 -0.00791699
 -0.00728478 -0.00686881 -0.00664737 -0.00646524 -0.00559181 -0.00469146
 -0.00422643 -0.00403053 -0.00977499 -0.00958962 -0.00943702 -0.01848611
 -0.01967668 -0.02025778 -0.02049158 -0.00166656 -0.00309443 -0.00330803
 -0.00345165 -0.00356323 -0.00374071 -0.00400407 -0.00453947 -0.00495278
 -0.00500746 -0.00503811 -0.00375553 -0.0044427  -0.00466862 -0.00472736
 -0.00350467 -0.00339595 -0.00324393 -0.00310622 -0.00301346 -0.00297268
 -0.00293916 -0.00278421 -0.00262904 -0.00255089 -0.00251329 -0.00367117
 -0.00364077 -0.00361333 -0.00517434 -0.00538645 -0.00549012 -0.00551884]


In [7]:
v1 = (V0) + B @ u1 + cal_disturbance(1)

print(v1)

[ 0.00455223  0.00615647  0.00267892  0.00133286  0.00188516 -0.00317437
 -0.0050308  -0.00304155  0.00213635  0.00157131  0.00333872  0.01072389
  0.01286148  0.01238196  0.01310466  0.0028412   0.00060885  0.00074338
 -0.00196461 -0.00262333 -0.00046611  0.00429481  0.00308241 -0.0014214
 -0.00319839 -0.00383981 -0.00261055 -0.00026378 -0.00199204  0.00454467
  0.00418712  0.00310049  0.00466845]


### Dynamics under strongly stable controller

In [8]:
scaling = 1
vol_list = []
action_list = []

v1 = (V0) + B @ u1 + cal_disturbance(1)

for t in range(500):
    v1 = v1 + B @ KK @ v1 + cal_disturbance(t+2) * scaling
    vol_list.append(max(v1) + 11)
    action_list.append(sum(np.abs(x) for x in KK @ v1))

print(vol_list)

print(max(vol_list))
print(sum(action_list))

# fail faced with large disturabnces

[11.004875153986985, 11.005570806978499, 11.003210674083123, 11.004105025652207, 11.003893774802787, 11.006239260546778, 11.001455252821966, 11.003444168911365, 11.003316703987053, 11.004684837456486, 11.001623131611195, 11.00295798937176, 11.00298355460616, 11.000209944428013, 11.013084361889298, 11.002579138056412, 11.002628999691007, 11.00122051174966, 11.004475546339922, 11.00233579043286, 11.002305669302949, 11.001085226115569, 11.004827886060355, 11.002200142734015, 11.002028871858702, 11.008122975230355, 11.000655231774665, 11.002040438757415, 11.001905343916432, 11.007978331436675, 11.00062061922974, 11.001785751423647, 11.001778398733027, 11.009236207187271, 11.000430044714888, 11.001615828737512, 11.0017040686881, 11.001049304024162, 11.003352491425016, 11.00157518243388, 11.001510424440736, 11.000789246874515, 11.005139782987788, 11.001466006319768, 11.001505943155081, 11.001025659313056, 11.002977249425198, 11.001497701830175, 11.001412111752755, 11.010260309002414, 11.0001

### Core algorithm

In [9]:
# Function to compute the power of a matrix
def matrix_power(matrix, power):
    result = tf.eye(matrix.shape[0], dtype=matrix.dtype)  # Start with the identity matrix
    for _ in range(power):
        result = tf.linalg.matmul(result, matrix)
    return result

In [10]:
def cal_derivative1(TT, MM_list, HH):
    """
    Input: time, controller parameters at last step, truncated length
    Output: derivative values of controller parameters
    """
    variables_list = []
    
    # Define the variables
    for i in range(HH):
        var_name = f'MM{i}'
        variables_list.append(tf.Variable(MM_list[i], name=var_name))

    # define constant values
    w_list = {key: None for key in range(2 * HH + 1)}

    for i in range(2 * HH + 1):
    # set w = 0 if time < 0 
        if (TT - 2 * HH + i) <= 0:
            w_list[i] = tf.constant(np.zeros(A.shape[0]), dtype=tf.float64)
            w_list[i] = tf.reshape(w_list[i], [A.shape[0], 1])
        else:
            w_list[i] = tf.constant(cal_disturbance(TT - 2 * HH + i) * scaling, dtype=tf.float64)  # enlarge disturbance
            w_list[i] = tf.reshape(w_list[i], [A.shape[0], 1])

    A_tf = tf.constant(A, dtype=tf.float64)
    B_tf = tf.constant(B, dtype=tf.float64)
    KK_tf = tf.constant(KK, dtype=tf.float64)
    zero_m = np.zeros((A.shape[0], A.shape[1]))

    # Define f
    def complex_matrix_function(MM):
        fai_list = {key: None for key in range(2 * HH + 1)}
        for i in range(2 * HH + 1):
            if i == 0:
                fai_list[i] = matrix_power((A_tf + B_tf @ KK_tf), i)
            if 1 <= i <= HH:
                fai_list[i] = matrix_power((A_tf + B_tf @ KK_tf), i)  
                #temp = tf.constant(zero_m, dtype=tf.float64)
                for j in range(i):
                    fai_list[i] += matrix_power((A_tf + B_tf @ KK_tf), j) @ B_tf @ MM[i-j-1]
                #fai_list[i] = fai_list[i] + temp
            if i >= (HH + 1):
                temp = tf.constant(zero_m, dtype=tf.float64)
                for j in range(HH + 1):
                    if 1 <= (i - j) <= HH:
                        temp += matrix_power((A_tf + B_tf @ KK_tf), j) @ B_tf @ MM[i-j-1]
                fai_list[i] = temp

        # surrogate state
        yyt = fai_list[0] @ w_list[2 * HH]

        for k in range(1, 2 * HH + 1):
            yyt += fai_list[k] @ w_list[2 * HH - k]
            
        # surrogate action
        vvt = KK_tf @ yyt

        for k in range(1, HH+1):
            vvt += MM[k-1] @ w_list[2 * HH - k]

    
        return 5 * tf.reduce_sum(tf.math.log(yyt + 0.55)) + 5 * tf.reduce_sum(tf.math.log(0.55 - yyt)) + \
            0.5 * tf.reduce_sum(tf.math.log(vvt + 0.01)) + 0.5 *  tf.reduce_sum(tf.math.log(0.01 - vvt))
    # choose quadratic function
    #1 * tf.transpose(yyt) @ yyt  + 10 * tf.transpose(vvt) @ vvt

    # Create a GradientTape context
    with tf.GradientTape() as tape:
        # Watch the variables
        tape.watch(variables_list)
        # Compute the function
        z = complex_matrix_function(variables_list)

    # Compute the gradient of z with respect to X and Y
    d_list = tape.gradient(z, variables_list)

    d_output = []
    for hh in range(HH):
        d_output.append(d_list[hh].numpy()) 

    return d_output


### Alg with incorporation the lastest info

In [11]:
def cal_derivative2(TT, MM_list, HH):
    """
    Input: time, controller parameters at last step, truncated length
    Output: derivative values of controller parameters
    """
    variables_list = []
    
    # Define the variables
    for i in range(HH):
        var_name = f'MM{i}'
        variables_list.append(tf.Variable(MM_list[i], name=var_name))

    # define constant values
    w_list = {key: None for key in range(2 * HH + 1)}

    for i in range(2 * HH + 1):
    # set w = 0 if time < 0 
        if (TT - 2 * HH + i + 1) <= 0:
            w_list[i] = tf.constant(np.zeros(A.shape[0]), dtype=tf.float64)
            w_list[i] = tf.reshape(w_list[i], [A.shape[0], 1])
        else:
            w_list[i] = tf.constant(cal_disturbance(TT - 2 * HH + i + 1) * scaling, dtype=tf.float64)
            # incorporate the lastest info
            w_list[i] = tf.reshape(w_list[i], [A.shape[0], 1])

    A_tf = tf.constant(A, dtype=tf.float64)
    B_tf = tf.constant(B, dtype=tf.float64)
    KK_tf = tf.constant(KK, dtype=tf.float64)
    zero_m = np.zeros((A.shape[0], A.shape[1]))

    # Define f
    def complex_matrix_function(MM):
        fai_list = {key: None for key in range(2 * HH + 1)}
        for i in range(2 * HH + 1):
            if i == 0:
                fai_list[i] = matrix_power((A_tf + B_tf @ KK_tf), i)
            if 1 <= i <= HH:
                fai_list[i] = matrix_power((A_tf + B_tf @ KK_tf), i)  
                #temp = tf.constant(zero_m, dtype=tf.float64)
                for j in range(i):
                    fai_list[i] += matrix_power((A_tf + B_tf @ KK_tf), j) @ B_tf @ MM[i-j-1]
                #fai_list[i] = fai_list[i] + temp
            if i >= (HH + 1):
                temp = tf.constant(zero_m, dtype=tf.float64)
                for j in range(HH + 1):
                    if 1 <= (i - j) <= HH:
                        temp += matrix_power((A_tf + B_tf @ KK_tf), j) @ B_tf @ MM[i-j-1]
                fai_list[i] = temp

        # surrogate state
        yyt = fai_list[0] @ w_list[2 * HH]

        for k in range(1, 2 * HH + 1):
            yyt += fai_list[k] @ w_list[2 * HH - k]
            
        # surrogate action
        vvt = KK_tf @ yyt

        for k in range(HH):
            vvt += MM[k] @ w_list[2 * HH - k]

    
        return 5 * tf.reduce_sum(tf.math.log(yyt + 0.55)) + 5 * tf.reduce_sum(tf.math.log(0.55 - yyt)) + \
            0.5 * tf.reduce_sum(tf.math.log(vvt + 0.01)) + 0.5 *  tf.reduce_sum(tf.math.log(0.01 - vvt))
    # choose quadratic function
    # 1 * tf.transpose(yyt) @ yyt  + 10 * tf.transpose(vvt) @ vvt  results: 0.182

    # Create a GradientTape context
    with tf.GradientTape() as tape:
        # Watch the variables
        tape.watch(variables_list)
        # Compute the function
        z = complex_matrix_function(variables_list)

    # Compute gradient
    d_list = tape.gradient(z, variables_list)

    d_output = []
    for hh in range(HH):
        d_output.append(d_list[hh].numpy()) 

    return d_output


### Evolution under such controller

In [12]:
def disturbance_update(TT, HH):
    """
    w_list is from early to present, i.e., w_list[-1] = w_t, w_list[-1] = w_{t-1}
    contain 2H +1 in total
    """
    w_list = {key: None for key in range(2 * HH + 1)}

    for i in range(2 * HH + 1):
        # set w = 0 if time < 0 
        if (TT - 2 * HH + i) <= 0:
            w_list[i] = np.zeros(A.shape[0])
        else:
            w_list[i] = cal_disturbance(TT - 2 * HH + i) * scaling

    return w_list

In [13]:
HH = 15

M_list = {key: np.vstack([np.eye(A.shape[0]) * np.power(1, key) * 0.02, 
                        np.eye(A.shape[0]) * np.power(1, key) * 0.02]) for key in range(HH)}


# handle sudden disturbance
# # M_list[1] = np.vstack([np.eye(A.shape[0]) * 0.1, 
# #                         np.eye(A.shape[0]) * 0.1])


vol_list2 = []
action_list2 = []
v1 = V0 + B @ KK @ V0 + cal_disturbance(1) * scaling
eta = 0.00001 


for t in range(1, 300):
    """
    Dynamics
    """
    # get latest disturbance
    w_list = disturbance_update(t, HH)

    # compute control action
    ctrl = np.zeros(B.shape[1])
    for i in range(0, HH):
        ctrl += M_list[i] @ w_list[2 * HH - i]

    #print("Max Additional action is {}".format(np.max(KK @ v1 + ctrl)))
    
    v1 = v1 + B @ (KK @ v1 + ctrl) + cal_disturbance(t+1) * scaling
    vol_list2.append(max(v1) + 11)
    action_list2.append(sum(np.abs(x) for x in KK @ v1 + ctrl))
    
    M_list_d = cal_derivative2(t, M_list, HH)
    #update controller parameters
    for hh in range(HH):
        M_list[hh] = M_list[hh] - eta * M_list_d[hh]
        
    print(max(v1))

    
print(vol_list2)

print(max(vol_list2))
#print(sum(action_list2))


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-06-05 12:46:38.085471: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-05 12:46:38.086387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


0.00486440586981477
0.0055628012929945965
0.0040483117507585705
0.004148973788937921
0.003981604881421836
0.0063528860487072954
0.003181383651726579
0.003630464727615821
0.0035253991447127208
0.0049197856534873895
0.0030208820178785623
0.0032455167229287736
0.0032751209204806026
0.00026973723511544527
0.006420068368055774
0.0030301727195428917
0.0030374016687859736
0.0011696965869358329
0.0033284817710570024
0.0027372752209313344
0.0026903101552058126
0.0008928205894594434
0.003254270309319866
0.0025222044129042283
0.0023939649046446215
0.007284066082142102
0.001162919025047461
0.002335665712282917
0.0021826989302234706
0.015387381877984902
0.0011676122278470066
0.00211505094320944
0.0020437118145965093
0.011239351165047196
0.0010776731407878936
0.00197173185162248
0.002010833807185155
0.0023881204024477623
0.002643893847690257
0.0019677383341613797
0.0019768750942862168
0.0004005941633640659
0.003353966260850787
0.0018968307795543648
0.0019515631024858233
0.0006233311472261971
0.00248

In [14]:
data = {
    'Wo': vol_list[150:180],
    'W': vol_list2[150: 180]
}

# Step 2: Create the DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


           Wo          W
0   10.999691  10.999926
1   11.001395  11.000576
2   11.000927  11.000015
3   10.999877  10.998563
4   11.003563  11.000446
5   11.187461  11.181505
6   11.002506  11.149070
7   11.003211  11.163598
8   11.003669  11.162981
9   11.001100  11.162277
10  10.990234  11.060444
11  11.101804  11.165208
12  10.981894  10.995456
13  11.002709  11.002565
14  11.002074  10.998405
15  11.000056  10.998363
16  11.003524  10.996844
17  11.002007  11.046922
18  11.001141  10.994387
19  11.003455  11.064150
20  10.999964  11.013536
21  11.001224  10.981253
22  11.000808  10.987116
23  10.999453  10.982995
24  11.007309  10.979895
25  11.001350  10.984637
26  11.000866  10.991301
27  11.004728  10.978517
28  10.999791  11.008025
29  11.000820  11.001859
